<a href="https://colab.research.google.com/github/higor-gomes93/mce_pesquisa_geicon/blob/main/MCE_Estendido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo de Geração de Mapas Conceituas Estendidos
---
Esse algoritmo tem por função a criação automática de Mapas Conceituas Estendidos, partindo de uma base de dados importada do Google Planilhas. Os dados foram obtidos por meio do formulário aplicado aos respondentes.

### 1) Importação das Bibliotecas
Todos as bibliotecas e pacotes necessários para o desenvolvimento do algoritmo.

In [129]:
# Importando as bibliotecas padrão
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rd

In [130]:
# Importando os pacotes necessários para conectar com o Google Drive e o Google Planilhas
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2) Geração da Base de Dados
Os dados são importados diretamente do Google Planilhas e convertidos em um DataFrame.

In [131]:
# Importando os dados do Google Planilhas
gc = gspread.authorize(GoogleCredentials.get_application_default())
planilha_de_dados = gc.open('[Formulários] Base de Dados Bruta').sheet1
rows = planilha_de_dados.get_all_values()

In [132]:
# Criação do objeto do tipo DataFrame
dataset = pd.DataFrame.from_records(rows)

# Transformando a primeira linha em cabeçalho
new_header = dataset.iloc[0]
dataset = dataset[1:]
dataset.columns = new_header

# Resetando o index
dataset = dataset.reset_index(drop = True)

# Transformando os dados nulos
dataset['Conceito Causa'].replace('', np.nan, inplace=True)
dataset['Conceito Efeito'].replace('', np.nan, inplace=True)
dataset['Linha CC'].replace('', np.nan, inplace=True)
dataset['Coluna CC'].replace('', np.nan, inplace=True)
dataset['Linha CE'].replace('', np.nan, inplace=True)
dataset['Coluna CE'].replace('', np.nan, inplace=True)

### 3) Tratamento dos Dados
Filtrar dados faltantes ou fora do padrão esperado para análise.

In [133]:
# Contando o total de respostas por aluno
valores_unicos = dataset.groupby(['Nome']).count()

In [134]:
# Selecionando apenas os que tiveram 5 respostas em todas as colunas
nomes_filtrados = valores_unicos.loc[(valores_unicos['Conceito Causa'] == 5) & 
                           (valores_unicos['Conceito Efeito'] == 5) &
                           (valores_unicos['Linha CC'] == 5) &
                           (valores_unicos['Coluna CC'] == 5) &
                           (valores_unicos['Linha CE'] == 5) &
                           (valores_unicos['Coluna CE'] == 5)]

nomes = list(nomes_filtrados.index)

In [135]:
# Filtrando o dataset
dataset = dataset.loc[dataset['Nome'].isin(nomes)]
dataset = dataset.reset_index(drop = True)
dataset_coordenadas = dataset.copy()
dataset_geral = dataset.copy()

In [136]:
# Tratando as strings
conceito_causa = list(dataset['Conceito Causa'])
conceito_causa = [i.strip().split(" ") for i in conceito_causa]
conceito_causa = ["\n".join(i) for i in conceito_causa]

conceito_efeito = list(dataset['Conceito Efeito'])
conceito_efeito = [i.strip().split(" ") for i in conceito_efeito]
conceito_efeito = ["\n".join(i) for i in conceito_efeito]

dataset['Conceito Causa'] = conceito_causa
dataset['Conceito Efeito'] = conceito_efeito

### 4) Criação dos Dicionários
Conversão dos atributos de linha e coluna em coordenadas cartesianas, a fim de usar o pano de fundo gerado previamente para o Mapa Conceitual Estendido.

In [137]:
# Criando os dicionários de correspondências das coordenadas para a imagem png
dicionario_linha_png = {
    'Controlável': (0, 300),
    'Penumbra': (300, 599),
    'Não Controlável': (599, 899)
}

dicionario_coluna_png = {
    'Não Controlável': (0, 600),
    'Penumbra': (600, 1199),
    'Controlável': (1199, 1799)
}

# Definindo uma margem
margem = 20

### 5) Imagem de Fundo
Carregando o pano de fundo para a construção do Mapa Conceitual Estendido.

In [138]:
fundo = plt.imread('/content/drive/Shareddrives/GEICon - Pessoas/Ferramentas/[Ferramentas] Formulário Tipos Psicológicos/[Ferramentas] Base de Dados/MCE.png')
plt.figure(figsize=(19.260416667, 9.8854166667))

<Figure size 1386.75x711.75 with 0 Axes>

<Figure size 1386.75x711.75 with 0 Axes>

### 6) Função de Geração dos Mapas Conceituais Estendidos
Função principal do algoritmo, cujo objetivo é criar um mapa conceitual estendido para cada um dos respondentes, e armazená-los em um diretório pré-estabelecido do Google Drive.

In [139]:
def criacao_mce(aluno):
    # Filtrando o dataset
    dataset_aluno = dataset.loc[dataset['Nome'] == aluno]
    
    # Separando conceito causa e conceito efeito
    lista_conceito_causa_aluno = list(dataset_aluno['Conceito Causa'])
    posicao_conceito_causa_aluno = dataset_aluno[['Coluna CC', 'Linha CC']]
    lista_posicao_conceito_causa_aluno = [tuple(x) for x in posicao_conceito_causa_aluno.to_numpy()]
    lista_conceito_efeito_aluno = list(dataset_aluno['Conceito Efeito'])
    posicao_conceito_efeito_aluno = dataset_aluno[['Coluna CE', 'Linha CE']]
    lista_posicao_conceito_efeito_aluno = [tuple(x) for x in posicao_conceito_efeito_aluno.to_numpy()]

    # Criação das coordenadas
    lista_coordenadas_conceito_causa_aluno = [(rd.randint(dicionario_coluna_png[i[0]][0]+margem, dicionario_coluna_png[i[0]][1]-margem), 
                                               rd.randint(dicionario_linha_png[i[1]][0]+margem, dicionario_linha_png[i[1]][1]-margem)) for i in lista_posicao_conceito_causa_aluno]
    lista_coordenadas_conceito_efeito_aluno = [(rd.randint(dicionario_coluna_png[i[0]][0]+margem, dicionario_coluna_png[i[0]][1]-margem), 
                                               rd.randint(dicionario_linha_png[i[1]][0]+margem, dicionario_linha_png[i[1]][1]-margem)) for i in lista_posicao_conceito_efeito_aluno]
    
    # Criando os vértices e arestas do grafo
    vertices_aluno = lista_conceito_causa_aluno
    vertices_aluno.extend(lista_conceito_efeito_aluno)
    arestas_aluno = [(lista_conceito_causa_aluno[i], lista_conceito_efeito_aluno[i]) for i in range(5)]
    
    # Criando o vetor de posições
    posicoes_conceito_causa = {lista_conceito_causa_aluno[i]: lista_coordenadas_conceito_causa_aluno[i] for i in range(5)}
    posicoes_conceito_efeito = {lista_conceito_efeito_aluno[i]: lista_coordenadas_conceito_efeito_aluno[i] for i in range(5)}
    posicoes = posicoes_conceito_causa
    posicoes.update(posicoes_conceito_efeito)

    # Inicializando um grafo direcionado
    grafo = nx.DiGraph()

    # Adicionando os vértices e arestas
    grafo.add_nodes_from(vertices_aluno)
    grafo.add_edges_from(arestas_aluno)

    # Gerando o grafo
    plt.figure(figsize=(19.260416667, 9.8854166667))
    plot = nx.draw_networkx_nodes(grafo, posicoes, node_shape='o', node_color = 'white', node_size = 300)
    plot = nx.draw_networkx_edges(grafo, posicoes, edgelist = arestas_aluno)
    lot = nx.draw_networkx_labels(grafo, posicoes, horizontalalignment='center', verticalalignment='top')
    plt.imshow(fundo, aspect='auto')
    plt.savefig('/content/drive/Shareddrives/GEICon - Pessoas/Ferramentas/[Ferramentas] Formulário Tipos Psicológicos/[Ferramentas] Base de Dados/[Ferramentas] Mapas Conceituais/'+aluno+'.png', bbox_inches = 'tight', pad_inches = 0)
    plt.close()
    

    return

### 7) Chamada da Função
Aplicar a função de criação dos Mapas Conceituais Estendidos para toda a lista dos respondentes.

In [140]:
'''
for i in nomes:
    criacao_mce(i)
'''

'\nfor i in nomes:\n    criacao_mce(i)\n'

### 8) Criação dos Dicionários HTML
Conversão dos atributos de linha e coluna em coordenadas cartesianas, a fim de usar o pano de fundo gerado previamente para o Mapa Conceitual Estendido no formato HTML, gerado no Google Sheets.

In [141]:
# Criando os dicionários de correspondências das coordenadas para a imagem png
dicionario_linha_html = {
    'Controlável': (0, 210),
    'Penumbra': (210, 419),
    'Não Controlável': (419, 629)
}

dicionario_coluna_html = {
    'Não Controlável': (0, 420),
    'Penumbra': (420, 840),
    'Controlável': (840, 1260)
}

# Definindo uma margem
margem = 20

### 9) Criação dos DataFrame Convertido
Substituição dos atributos categóricos das linhas e colunas por coordenadas.

In [142]:
# Extraindo as listas das posições categóricas para conceitos causa e efeito
posicao_conceito_causa = dataset[['Coluna CC', 'Linha CC']]
lista_posicao_conceito = [tuple(x) for x in posicao_conceito_causa.to_numpy()]
posicao_conceito_efeito = dataset[['Coluna CE', 'Linha CE']]
lista_posicao_conceito_efeito = [tuple(x) for x in posicao_conceito_efeito.to_numpy()]

# Transformando em coordenadas cartesianas
lista_coordenadas_conceito_causa = [(rd.randint(dicionario_coluna_html[i[0]][0]+margem, dicionario_coluna_html[i[0]][1]-margem), 
                                            rd.randint(dicionario_linha_html[i[1]][0]+margem, dicionario_linha_html[i[1]][1]-margem)) for i in lista_posicao_conceito]
x_conceito_causa = [i[0] for i in lista_coordenadas_conceito_causa]
y_conceito_causa = [i[1] for i in lista_coordenadas_conceito_causa]


lista_coordenadas_conceito_efeito = [(rd.randint(dicionario_coluna_html[i[0]][0]+margem, dicionario_coluna_html[i[0]][1]-margem), 
                                            rd.randint(dicionario_linha_html[i[1]][0]+margem, dicionario_linha_html[i[1]][1]-margem)) for i in lista_posicao_conceito_efeito]
x_conceito_efeito = [i[0] for i in lista_coordenadas_conceito_efeito]
y_conceito_efeito = [i[1] for i in lista_coordenadas_conceito_efeito]

# Criando o novo dataset
dataset_coordenadas['Coluna CC'] = x_conceito_causa
dataset_coordenadas['Linha CC'] = y_conceito_causa
dataset_coordenadas['Coluna CE'] = x_conceito_efeito
dataset_coordenadas['Linha CE'] = y_conceito_efeito
dataset_coordenadas

,Nome,Conceito Causa,Conceito Efeito,Linha CC,Coluna CC,Linha CE,Coluna CE,Data
0,Juan Costa Dos Santos,Deixar um legado para o próximo,o desejo de vencer,147,1238,67,1106,
1,Juan Costa Dos Santos,Crescimento,Produtividade dos colaboradores,82,1233,89,937,
2,Juan Costa Dos Santos,Inveja,Stress nas relações sociais,135,1047,65,1089,
3,Juan Costa Dos Santos,Investimento no funcionario,Crescimento,164,1085,90,1111,
4,Juan Costa Dos Santos,Tentar se beneficiar diminuindo o outro,Inveja,60,1065,176,869,
...,...,...,...,...,...,...,...,...
405,Marcelli Roberta Sarti,SUCESSO,o desejo de vencer,116,1124,88,1100,
406,Marcelli Roberta Sarti,DESORGANIZAÇÃO,Stress nas relações sociais,165,869,94,916,
407,Marcelli Roberta Sarti,COMUNICAÇÃO,Produtividade dos colaboradores,133,869,171,900,
408,Marcelli Roberta Sarti,ESFORÇO,SUCESSO,182,1062,156,860,


In [143]:
from gspread_dataframe import set_with_dataframe

nome_planilha1 = '[Formulários] Base de Dados - Coordenadas'
planilha1 = gc.open(nome_planilha1).sheet1
set_with_dataframe(planilha1, dataset_coordenadas, include_index = False, include_column_header = True, resize = False)

nome_planilha2 = '[Formulários] Base de Dados - Geral'
planilha2 = gc.open(nome_planilha2).sheet1
set_with_dataframe(planilha2, dataset_geral, include_index = False, include_column_header = True, resize = False)